Code by clefourrier

In [1]:
from datasets import load_dataset
dataset_en = load_dataset('bigscience-historical-texts/hipe2020', 'en')
dataset_de = load_dataset('bigscience-historical-texts/hipe2020', 'de')
dataset_fr = load_dataset('bigscience-historical-texts/hipe2020', 'fr')

Reusing dataset hipe2020 (/home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/en/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0)


  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset hipe2020 (/home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/de/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset hipe2020 (/home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/fr/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0)


  0%|          | 0/2 [00:00<?, ?it/s]

# Data split

In [2]:
date_junction = range(1790, 1951, 20) 
period_dict_en = {i: [] for i in date_junction}
period_dict_de = {i: [] for i in date_junction}
period_dict_fr = {i: [] for i in date_junction}
date_counter = 0

for datasets, period_dict in [
    ([dataset_en["validation"]], period_dict_en),
    ([dataset_fr["validation"], dataset_fr["train"]], period_dict_fr),
    ([dataset_de["validation"], dataset_de["train"]], period_dict_de),
]:
    for dataset in datasets:
        dataset = dataset.sort("date")
        for item in dataset:
            # Our ranges are [start_year, end_year[
            if item['date'].year > 1949:
                date_counter = 0
                break
            while item['date'].year >= date_junction[date_counter + 1]:
                date_counter += 1

            period_dict[date_junction[date_counter]].append(item)
        date_counter = 0

Loading cached sorted indices for dataset at /home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/en/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0/cache-67a472637b33bd78.arrow
Loading cached sorted indices for dataset at /home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/fr/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0/cache-ce6e6f5f6136c5ce.arrow
Loading cached sorted indices for dataset at /home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/fr/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0/cache-2c360d7f024cf07a.arrow
Loading cached sorted indices for dataset at /home/cfourrie/.cache/huggingface/datasets/bigscience-historical-texts___hipe2020/de/1.0.0/da88d36532438bb0dd04d66712679916204f6ab903eaba3aec59c18ee4c685b0/cache-f6ec2e038cc64982.arrow
Loading cached sorted indices for dataset at /home/cfourrie/.cache/huggingface/d

# Latex table generation

In [3]:
for period in date_junction[:-1]:
    result = f"{period}-{period+20} & "
    for period_dict in [period_dict_en, period_dict_de, period_dict_fr]:
        period_list = period_dict[period]
        num_items = sum([len(i["NE_COARSE_LIT"]) for i in period_list])
        num_ner = sum(len(i["NE_COARSE_LIT"]) - i["NE_COARSE_LIT"].count(0) for i in period_list)
        if num_items == 0:
            result += f"{len(period_list)} & {num_items} & - & "
        else:
            result += f"{len(period_list)} & {num_items} & " + '{:2.1%}'.format(num_ner/num_items) + " & "
    print(result[:-2].replace("%", "\%") + "\\\\")

end_result = "Total & " 
for period_dict in [period_dict_en, period_dict_de, period_dict_fr]:
    num_items = sum([len(i["NE_COARSE_LIT"]) for period_list in period_dict.values() for i in period_list])
    num_ner = sum(len(i["NE_COARSE_LIT"]) - i["NE_COARSE_LIT"].count(0) for period_list in period_dict.values() for i in period_list)
    num_docs = sum([len(period_list) for period_list in period_dict.values()])
    end_result += f"{num_docs} & {num_items} & " + '{:2.1%}'.format(num_ner/num_items) + " & "
print(end_result[:-2].replace("%", "\%") + "\\\\")


1790-1810 & 459 & 4143 & 8.7\% & 743 & 6944 & 8.9\% & 707 & 8550 & 10.3\% \\
1810-1830 & 549 & 4697 & 9.5\% & 787 & 8534 & 6.2\% & 917 & 12440 & 14.1\% \\
1830-1850 & 476 & 3974 & 11.1\% & 1643 & 16212 & 6.7\% & 1028 & 11659 & 6.9\% \\
1850-1870 & 0 & 0 & - & 1832 & 16505 & 7.8\% & 948 & 10321 & 8.9\% \\
1870-1890 & 281 & 2202 & 6.0\% & 2161 & 18071 & 7.0\% & 2012 & 16272 & 9.6\% \\
1890-1910 & 597 & 4509 & 8.5\% & 1577 & 13097 & 10.8\% & 1976 & 16874 & 10.3\% \\
1910-1930 & 763 & 5499 & 9.6\% & 2356 & 18649 & 7.0\% & 3607 & 30403 & 9.4\% \\
1930-1950 & 78 & 520 & 15.6\% & 3440 & 25123 & 10.6\% & 4494 & 35962 & 10.5\% \\
Total & 3203 & 25544 & 9.3\% & 14539 & 123135 & 8.3\% & 15689 & 142481 & 10.0\% \\
